In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.metrics import accuracy_score, mean_squared_error
from math import sqrt
from keras.utils import to_categorical
import pandas as pd
from PIL import Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import mean_squared_error, r2_score
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error

In [11]:
import os

train_folder = r"preprocessed-boneage-training-dataset"
test_folder = r"boneage-test-dataset"

In [12]:
# Count the number of folders in the train and test directories
num_classes_train = len(os.listdir(train_folder))
num_classes_test = len(os.listdir(test_folder))


In [13]:
from keras.utils import to_categorical
import pandas as pd
from PIL import Image

# Load the metadata from the CSV file
csv_file_path = r"boneage-training-dataset.csv"
df = pd.read_csv(csv_file_path)

# Lists to store the image data and bone age labels
x_train = []
y_train = []

In [14]:
df['gender'] = df['male'].apply(lambda x: 'male' if x else 'female')
print(df['gender'].value_counts())

gender
male      6833
female    5778
Name: count, dtype: int64


In [15]:
df = df.drop('male', axis=1)



In [16]:
df

,id,boneage,gender
0,1377,180,female
1,1378,12,female
2,1379,94,female
3,1380,120,male
4,1381,82,female
...,...,...,...
12606,15605,50,female
12607,15606,113,female
12608,15608,55,female
12609,15609,150,male


In [17]:
# Loop through the rows in the DataFrame
for index, row in df.iterrows():
    image_id = row['id']
    image_filename = f"{image_id}.png"  # Assuming the image filename is stored as an integer 'id'
    bone_age = row['boneage']
    image_path = os.path.join(train_folder, image_filename)
    if os.path.isfile(image_path):
        try:
            # Open and preprocess the image
            img = Image.open(image_path)
            # Preprocess the image as needed (e.g., resize, normalization)
            img = np.array(img)
            img = img.astype("float32") / 255.0
            # Add the preprocessed image and bone age label to the lists
            x_train.append(img)
            y_train.append(bone_age)
        except Exception as e:
            print(f"Error processing image: {image_filename}, Error: {e}")

# Convert the lists to numpy arrays
x_train = np.array(x_train)#this has images as numbers
y_train = np.array(y_train)#tis has bone age in number

print(x_train)
print(y_train)

[[[0.7764706  0.67058825 0.30980393 ... 0.3137255  0.3137255  0.31764707]
  [1.         1.         0.50980395 ... 0.30980393 0.30980393 0.3137255 ]
  [1.         1.         0.96862745 ... 0.30588236 0.30588236 0.30588236]
  ...
  [0.16470589 0.16470589 0.16862746 ... 0.18431373 0.18431373 0.18431373]
  [0.16470589 0.16862746 0.16862746 ... 0.18431373 0.18431373 0.18431373]
  [0.16470589 0.16862746 0.16862746 ... 0.18431373 0.18431373 0.18431373]]

 [[0.2784314  0.2784314  0.2784314  ... 0.2784314  0.2784314  0.2784314 ]
  [0.2784314  0.2784314  0.2784314  ... 0.2784314  0.2784314  0.2784314 ]
  [0.2784314  0.2784314  0.2784314  ... 0.2784314  0.2784314  0.2784314 ]
  ...
  [0.2784314  0.2784314  0.2784314  ... 0.27058825 0.27058825 0.27058825]
  [0.2784314  0.2784314  0.2784314  ... 0.27058825 0.27450982 0.27450982]
  [0.2784314  0.2784314  0.2784314  ... 0.27450982 0.27450982 0.27450982]]

 [[1.         1.         1.         ... 0.36862746 0.25490198 0.18039216]
  [1.         1.      

In [18]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import mean_squared_error, r2_score
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error
import numpy as np

# Normalize the images to [0, 1]
x_train = x_train.astype("float32") / 255.0

# Reshape the data to match the CNN input shape (assuming grayscale images)
input_shape = x_train[0].shape + (1,)
x_train = x_train.reshape(-1, *input_shape)

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Build your CNN model architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))  # Output layer for regression (bone age)

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model with the validation data
history = model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_val, y_val))




# Predict on the validation set
y_val_pred = model.predict(x_val)

# Calculate validation metrics
mse_val = mean_squared_error(y_val, y_val_pred)
mae_val = mean_absolute_error(y_val, y_val_pred)
r2_val = r2_score(y_val, y_val_pred)

# Print validation metrics
print("Validation MSE:", mse_val)
print("Validation MAE:", mae_val)
print("Validation R2:", r2_val)


c:\Users\Rohit Kumar\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 116s 363ms/step - loss: 3439.5803 - val_loss: 1810.1487
Epoch 2/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 112s 355ms/step - loss: 1702.0905 - val_loss: 1806.4570
Epoch 3/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 112s 354ms/step - loss: 1724.5748 - val_loss: 1761.6134
Epoch 4/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 113s 356ms/step - loss: 1684.4677 - val_loss: 2102.8213
Epoch 5/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 112s 354ms/step - loss: 1734.4781 - val_loss: 1719.4161
Epoch 6/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 112s 353ms/step - loss: 1675.5520 - val_loss: 1807.9786
Epoch 7/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 114s 360ms/step - loss: 1694.7572 - val_loss: 1684.1202
Epoch 8/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 112s 355ms/step - loss: 1670.9261 - val_loss: 1670.9980
Epoch 9/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 112s 354ms/step - loss: 1588.1448 - val_loss: 1659.6096
Epoch 10/10
316/316 ━━━━━━━━━━━━━━━━━━━━ 112s 355ms/step - loss: 1597.8241 - val_loss: 1677.0642
79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step

In [19]:
model.save('cnn_model.h5')

In [30]:
# Define the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=1,
    directory='my_dir',
    project_name='cnn_hyperparameter_tuning'
)

# Search for the best hyperparameters
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))


Trial 5 Complete [00h 18m 41s]
val_loss: 1731.088623046875

Best val_loss So Far: 1711.6434326171875
Total elapsed time: 01h 16m 28s


In [31]:
# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Get the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
print(best_hyperparameters.values)


Best Hyperparameters:
{'filters': 128, 'kernel_size': 5, 'pool_size': 3, 'units': 64, 'optimizer': 'adam'}


c:\Users\Rohit Kumar\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [33]:
# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]


In [34]:
best_model.compile(optimizer=best_model.optimizer, 
                   loss='mean_squared_error', 
                   metrics=['mae'])


In [37]:
best_history = best_model.fit(x_train, y_train, 
                         validation_data=(x_val, y_val), 
                         epochs=10, 
                         batch_size=32)


In [ ]:

# Save the best model
best_model.save('best_cnn_model.h5')
